In [1]:
# For python2/3 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

def onehot_tensor(t, num_classes):
    out = np.zeros((t.shape[0], t.shape[1], num_classes))
    for batch in range(t.shape[0]):
        for row, col in enumerate(t[batch]):
            out[batch, row, col] = 1
    return out

def onehot_vector(t, num_classes):
    out = np.zeros((t.shape[0], num_classes))
    for batch in range(t.shape[0]):
        out[batch, t[batch]] = 1
    return out

# Conditional random fields

# Definition

We define the conditional random field for label sequence $y_1,\ldots,y_n$:
$$
p(y|h) = \frac{1}{Z} \prod_i \exp f(y_i,h_i) \prod_{i=1}^{n-1} \exp g(y_i,y_{i+1},h_i) 
$$
The $h_i$ is the $i$th output of a recurrent (potentially bi-direncetional) RNN. $f(y_i,h_i)$ is the standard 
feed-forward pre-softmax output and $g(y_i,y_{i+1},h_i)$ is the random field interaction term. If we have $c$ 
classes we can implement $g(y_i,y_{i+1},h_i)$ as a linear output network with $c^2$ outputs.   

The normalization constant is
$$
Z = \sum_{y_1} \ldots \sum_{y_n} \prod_i \exp f(y_i,h_i) \prod_{i=1}^{n-1} \exp g(y_i,y_{i+1}h_i) \ .
$$

## Simple example 

Let us make a simple example:

In [7]:
b = 3
n = 50 # length of sequence
c = 2 # number of classes

g = np.zeros((b, n-1, c, c)) # independent variables
f = np.ones((b, n, c)) # this should lead to equal probability for both classes 

# define some example data that we will use below to calculate the log likelihood
y = np.zeros([b, n], dtype=np.int32) # all label belong to class 1
y_hot = onehot_tensor(y, c)
print(y.shape)
print(y_hot.shape)

(3, 50)
(3, 50, 2)


# Inference

We can handle inference that is computing $Z$, marginals
$$
p(y_i|h) = \frac{1}{Z} \sum_{y_1} \ldots \sum_{y_{i-1}} \sum_{y_{i+1}} \ldots \sum_{y_n} \prod_i \exp f(y_i,h_i) \prod_{i=1}^{n-1} \exp g(y_i,y_{i+1},h_i) 
$$
and the most probably output sequence
$$
{\rm argmax}_y p(y|h)
$$
by dynamic programming. The dynamic programming requires one forward and one backward pass to solve the two first tasks and a different forward and backward pass (the so-called Viterbi algorithm) for the last task.


# Forward recursion

See Bishop 8.4.1 for justification. Define $\mu_\alpha(y_i)$ (a vector of dimensionality the number of classes $c$) as the following recursion
\begin{align*}
\mu_\alpha(y_1) & = \exp f(y_1,h_1) \\
\mu_\alpha(y_i) & = \exp f(y_i,h_i) \sum_{y_{i-1}} \exp g(y_{i-1},y_{i},h_{i-1}) \mu_\alpha(y_{i-1})
\end{align*}
In practice the sum will be a $c$-dimensional matrix multiplication.
In the log-domain $\nu_\alpha(y_i) \equiv \log \mu_\alpha(y_i)$ we have
\begin{align*}
\nu_\alpha(y_1) & = f(y_1,h_1) \\
\nu_\alpha(y_i) & = f(y_i,h_i) + \log \sum_{y_{i-1}} \exp ( g(y_{i-1},y_{i},h_{i-1}) + \nu_\alpha(y_{i-1}) ) \ . 
\end{align*}
We might need a bit of max book-keeping to make this stable but let us forget that in the first implementation. 
We can probably handle this by introducing a ${\rm log\_sum\_exp}$ function: ${\rm log\_sum\_exp}(x)=x_{{\rm max}} + \log \sum_k \exp(x_m - x_{{\rm max}}$, where $x_{{\rm max}} =\max_k x_k$. 

In [61]:
def log_sum_exp(z, axis=1):
    zmax = np.max(z, axis)
    return zmax + np.log( np.sum( np.exp( z - np.expand_dims(zmax, axis)), axis))
     
def forward_step(nu, f, g, class_dimension):
    return f + log_sum_exp(g + np.expand_dims(nu, class_dimension+1), axis=class_dimension)

def forward_pass(f, g, time_major=False):
    if not time_major:
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    nu = np.zeros(np.shape(f))
    nu[0] = f[0]
    sequence_length = f.shape[0]
    class_dimension = 1
    for index in range(1, sequence_length):
        fs = forward_step(nu[index-1], f[index], g[index-1], class_dimension)
        nu[index] = fs
    if not time_major:
        nu = np.transpose(nu, [1, 0, 2])
    return nu

nu_alp = forward_pass(f, g)
print(nu_alp.shape)
print(nu_alp[0].T)

(3, 50, 2)
[[  1.           2.69314718   4.38629436   6.07944154   7.77258872
    9.4657359   11.15888308  12.85203026  14.54517744  16.23832463
   17.93147181  19.62461899  21.31776617  23.01091335  24.70406053
   26.39720771  28.09035489  29.78350207  31.47664925  33.16979643
   34.86294361  36.55609079  38.24923797  39.94238515  41.63553233
   43.32867951  45.02182669  46.71497388  48.40812106  50.10126824
   51.79441542  53.4875626   55.18070978  56.87385696  58.56700414
   60.26015132  61.9532985   63.64644568  65.33959286  67.03274004
   68.72588722  70.4190344   72.11218158  73.80532876  75.49847594
   77.19162313  78.88477031  80.57791749  82.27106467  83.96421185]
 [  1.           2.69314718   4.38629436   6.07944154   7.77258872
    9.4657359   11.15888308  12.85203026  14.54517744  16.23832463
   17.93147181  19.62461899  21.31776617  23.01091335  24.70406053
   26.39720771  28.09035489  29.78350207  31.47664925  33.16979643
   34.86294361  36.55609079  38.24923797  39.94238

# Backward recursion

The backward recursion follows the same reasoning (details omitted): 
\begin{align*}
\nu_\beta(y_n) & = 0 \\
\nu_\beta(y_i) & = \log \sum_{y_{i+1}} \exp ( f(y_{i+1},h_{i+1}) + g(y_{i},y_{i+1},h_{i}) + \nu_\alpha(y_{i+1}) ) \ . 
\end{align*}


In [62]:
def backward_step(nu, f, g, class_dimension):
    return log_sum_exp(np.expand_dims(f + nu, class_dimension) + g, axis=class_dimension+1)

def backward_pass(f, g, time_major=False):
    if not time_major:
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    nu = np.zeros(np.shape(f))
    sequence_length = f.shape[0]
    class_dimension = 1
    for index in range(1, sequence_length):
        nu[-index-1] = backward_step(nu[-index], f[-index],g[-index],
                                     class_dimension=class_dimension)
    if not time_major:
        nu = np.transpose(nu, [1, 0, 2])
    return nu

nu_bet = backward_pass( f, g)

print(nu_bet[0].T)

[[ 82.96421185  81.27106467  79.57791749  77.88477031  76.19162313
   74.49847594  72.80532876  71.11218158  69.4190344   67.72588722
   66.03274004  64.33959286  62.64644568  60.9532985   59.26015132
   57.56700414  55.87385696  54.18070978  52.4875626   50.79441542
   49.10126824  47.40812106  45.71497388  44.02182669  42.32867951
   40.63553233  38.94238515  37.24923797  35.55609079  33.86294361
   32.16979643  30.47664925  28.78350207  27.09035489  25.39720771
   23.70406053  22.01091335  20.31776617  18.62461899  16.93147181
   15.23832463  13.54517744  11.85203026  10.15888308   8.4657359
    6.77258872   5.07944154   3.38629436   1.69314718   0.        ]
 [ 82.96421185  81.27106467  79.57791749  77.88477031  76.19162313
   74.49847594  72.80532876  71.11218158  69.4190344   67.72588722
   66.03274004  64.33959286  62.64644568  60.9532985   59.26015132
   57.56700414  55.87385696  54.18070978  52.4875626   50.79441542
   49.10126824  47.40812106  45.71497388  44.02182669  42.3286

# Log likelihood

In training we have a sequence $y$ and we want to evaluate its probability. We have
$$
\log p(y|h) = \sum_i f(y_i,h_i) + \sum_{i=1}^{n-1}  g(y_i,y_{i+1},h_i) - \log Z
$$
Now the miracle happens because we can write $Z$ as
$$
Z = \sum_{y_i} \exp(\nu_\alpha(y_i) + \nu_\beta(y_i) )
$$
(This result holds for any choice of $i\in \{1,\ldots,n\}$.)


In [63]:
def logZ(nu_alp, nu_bet, index=0, time_major=False):
    if not time_major:
        nu_alp = np.transpose(nu_alp, [1, 0, 2])
        nu_bet = np.transpose(nu_bet, [1, 0, 2])
    class_dimension = 1
    return log_sum_exp(nu_alp[index]+nu_bet[index],
                       axis=class_dimension) 

def log_likelihood(y, f, g, nu_alp, nu_bet, mean_batch=True, time_major=False):
    if not time_major:
        y = np.transpose(y, [1, 0])
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    sequence_length = np.shape(f)[0]
    batch_size = np.shape(f)[1]
    f_term = np.zeros((batch_size))
    g_term = np.zeros((batch_size))
    z_term = np.zeros((batch_size))
    for index in range(sequence_length):
        f_term += f[index, np.arange(batch_size), y[index]] # f(y_i,h_i) term
    for index in range(sequence_length - 1):
        g_term += g[index, np.arange(batch_size), y[index + 1], y[index]] # g(y_i,y_i+1,h_i)
    z_term = logZ( nu_alp, nu_bet)
    log_like = f_term + g_term - z_term
    if mean_batch:
        log_like = np.mean(log_like)
    return log_like

def extract_values_vector_f(f, y):
    dim0, dim1, dim2 = f.shape
    idx0 = np.repeat(np.arange(dim0), dim1)
    idx1 = np.asarray([np.arange(dim1)] * dim0).reshape([-1])
    idx2 = y.reshape([-1])
    return f[idx0, idx1, idx2].reshape([dim0, dim1])

def extract_values_vector_g(g, y):
    g_seq_dim, batch_dim, c_dim1, c_dim2 = g.shape
    g_seq_idx = np.repeat(np.arange(g_seq_dim), batch_dim)
    g_batch_idx = np.asarray([np.arange(batch_dim)] * (g_seq_dim)).reshape([-1])
    y1 = y[g_seq_idx, g_batch_idx]
    y2 = y[g_seq_idx+1, g_batch_idx]
    john = g[g_seq_idx, g_batch_idx, y1, y2].reshape([g_seq_dim, batch_dim])
    return g[g_seq_idx, g_batch_idx, y1, y2].reshape([g_seq_dim, batch_dim])

def log_likelihood_vectorized(y, f, g, nu_alp, nu_bet, mean_batch=True, time_major=False):
    if not time_major:
        y = np.transpose(y, [1, 0])
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    f_term = np.sum(extract_values_vector_f(f, y), axis=0)
    g_term = np.sum(extract_values_vector_g(g, y), axis=0)
    z_term = logZ(nu_alp, nu_bet)
    log_like = f_term + g_term - z_term
    if mean_batch:
        log_like = np.mean(log_like)
    return log_like

print("log_likelihood")
print(log_likelihood(y, f, g, nu_alp, nu_bet))
print()
print("log_likelihood_vectorized")
print(log_likelihood_vectorized(y, f, g, nu_alp, nu_bet))
print()
print(n*np.log(0.5)) # compare to log likelihood for n independent variables with 0.5 probability

print(logZ( nu_alp, nu_bet)) # should give same log Z no matter what slice we use
print(logZ( nu_alp, nu_bet, 2))
print(logZ( nu_alp, nu_bet, n-1))

log_likelihood
-34.657359028

log_likelihood_vectorized
-34.657359028

-34.657359028
[ 84.65735903  84.65735903  84.65735903]
[ 84.65735903  84.65735903  84.65735903]
[ 84.65735903  84.65735903  84.65735903]


In [64]:
# The one-hot encoded way of calculating log_likelihood

def log_likelihood_hot(y, f, g, nu_alp, nu_bet, mean_batch=True, time_major=False):
    if time_major:
        y = np.transpose(y, [1, 0, 2])
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    f_term = np.sum(f*y, axis=(1, 2))
    y_i = np.expand_dims(y[:, :-1], axis=3)
    y_plus = np.expand_dims(y[:, 1:], axis=2)
    g_term = np.sum(g*y_i*y_plus, axis=(1,2,3))
    z_term = logZ(nu_alp, nu_bet)
    log_like = f_term + g_term - z_term
    if mean_batch:
        log_like = np.mean(log_like)
    return log_like
    
print("log_likelihood")
print(log_likelihood_hot(y_hot, f, g, nu_alp, nu_bet))

log_likelihood
-34.657359028


## Prediction

We will usually choose as out prediction the most probable label so we need to compute the marginal. Again the miracle (called dynamic programming) happens:
$$
p(y_i|h) = \frac{1}{Z} \exp(\nu_\alpha(y_i) + \nu_\beta(y_i) ) \ .
$$

In [65]:
print(nu_alp.shape)
print(nu_bet.shape)

(3, 50, 2)
(3, 50, 2)


In [66]:
def log_marginal(nu_alp, nu_bet, index=None, time_major=False):
    if not time_major:
        nu_alp = np.transpose(nu_alp, [1, 0, 2])
        nu_bet = np.transpose(nu_bet, [1, 0, 2])
    sequence_length = nu_alp.shape[0]
    if index is None:
        index=np.arange(sequence_length)
    r1 = nu_alp[index]
    r2 = nu_bet[index]
    r3 = np.expand_dims(logZ(nu_alp, nu_bet, time_major=True), axis=1)
    res = r1 + r2 - r3
    if not time_major:
        if len(res.shape) == 3:
            res = np.transpose(res, [1, 0, 2])
    return res

print(np.exp(log_marginal( nu_alp, nu_bet))[0].T)
print(np.exp(log_marginal( nu_alp, nu_bet, 0))[0].T)
print(np.exp(log_marginal( nu_alp, nu_bet, 2))[0].T)

[[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
 [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]]
[ 0.5  0.5]
[ 0.5  0.5]


# Viterbi decoding (max-sum algorithm)
This is useful when we are interested in the most probable sequence and not just the most probable classification at each position 
(think machine translation). This derivation is an adaptation of https://en.wikipedia.org/wiki/Viterbi_algorithm.  As a warm-up calculate
$$
\max_y \log p(y|h) = \max_{y_1}\cdots \max_{y_n} \left\{ \sum_i f(y_i,h_i) + \sum_{i=1}^{n-1}  g(y_i,y_{i+1},h_i) - \log Z \right\} \ .
$$
We can solve this with dynamic programming again by introducing
\begin{align*}
\mu(y_{i+1}) & = \max_{y_i}  \left\{ f(y_i,h_i) + g(y_i,y_{i+1},h_i) + \mu(y_{i}) \right\} \\
\mu(y_{1}) & = 0 \ .
\end{align*}
Now we simply get
$$
\max_y \log p(y|h) = \max_{y_n} \mu(y_{n}) - \log Z \ . 
$$
To prove this start from $\max_{y_1}$ in the original expression and introduce $\mu(y_2)$ and the remaining $\mu$s recursively. 

In order to find the most probable sequence we have to introduce a bit more bookkeeping. We immediately see that the the last symbol in the most probable sequence is ${\rm argmax}_{y_n} \mu(y_{n})$. But to get the second last symbol and so on requires that we keep track of the max value sequences for all possible configurations of $y_n$. We therefore introduce a variable that keeps the index of proceeding symbol for each setting of the $i+1$ symbol:
\begin{align*}
\nu_{i+1}(y_{i+1}) & = {\rm argmax}_{y_i} \left\{ g(y_i,y_{i+1},h_i) + \mu(y_{i}) \right\}
\end{align*}
Now we have the information for find the most probable sequence $s_1,\ldots,s_n$ from a backward recursion, $i=n-1,n-2,\ldots,1$:
\begin{align*}
s_n & = {\rm argmax}_{y_n} \mu(y_{n})\\ 
s_i & = \nu_{i+1}(y_{i+1}=s_{i+1}) \ .
\end{align*}

In [67]:
def forward_step_max(nu, f, g, class_dimension):
    return f + np.max(g + np.expand_dims(nu, class_dimension+1), axis=class_dimension)

def viterbi( f, g, time_major=False):
    if not time_major:
        f = np.transpose(f, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    sequence_length = np.shape(f)[0]
    class_dimension = 1
    nu = np.zeros(f.shape)
    nu_label = np.zeros(f.shape, dtype=np.int32)
    viterbi_seq = np.zeros(np.shape(f[:,:,0]), dtype=np.int32)
    nu[0] = f[0]
    for index in range(1, sequence_length):
        nu[index] = forward_step_max(nu[index-1], f[index], g[index-1], class_dimension)
        nu_label[index] = np.argmax(g[index-1] + np.expand_dims(nu[index-1], class_dimension+1), axis=class_dimension)
    viterbi_seq[-1] = np.argmax(nu[-1], axis=class_dimension)
    for index in range(1, sequence_length):
        #viterbi_seq[-index-1] = nu_label[-index,np.arange(f.shape[1]),viterbi_seq[-index]]
        viterbi_seq[-index-1] = np.sum( nu_label[-index] * onehot_vector(viterbi_seq[-index], np.shape(f)[2]), axis=1)
    if not time_major:
        viterbi_seq = np.transpose(viterbi_seq, [1, 0])
    # print(np.transpose(nu, [1, 0, 2]))
    return viterbi_seq #, np.max(nu[-1], axis=class_dimension)
        
print(viterbi( f, g))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]]


# Sampling

We will in some cases want to draw samples from $p(y|h)$. It could be that we wanted to generate alternative translations in a machine translation model or let $y$ be discrete latent variables in a generative model. We can write the joint using the chain rule 
$$
p(y|h) = p(y_1|y_2,\ldots,y_n,h)p(y_2|y_3,\ldots,y_n,h) \cdots p(y_n|h)
$$
and then sample the joint by sampling one term at a time starting from the last. The $p(y_n|h)$ we can sample from using the result for the forward pass: $p(y_n|h) \propto \exp{\mu_\alpha(y_n)}$ . To get the subsequent terms we observe that due to the chain structure $y_{i+1}$ ``blocks'' the dependence of the remaining variables: 
$$
p(y_i|y_{i+1},\ldots,y_n,h)=p(y_i|y_{i+1},h) \ .
$$
The conditional is always proportional to the joint:
$$
p(y_i|y_{i+1},h) \propto p(y_i,y_{i+1}|h) = \frac{1}{Z} 
\sum_{y_1} \ldots \sum_{y_{i-1}} \sum_{y_{i+1}} \ldots \sum_{y_n} 
\prod_i \exp f(y_i,h_i) \prod_{i=1}^{n-1} \exp g(y_i,y_{i+1},h_i) 
$$
Since $y_{i+1}$ is observed we can ignore dependence on $y_{i+2}$ and onwards. The marginalization over $y_1, \ldots, y_{i-1}$ is corresponds exactly to the computation in $\mu_\alpha$ so we get
\begin{align*}
p(y_i|y_{i+1},\ldots,y_n,h) & \propto \exp \mu_\alpha(y_i) + g(y_i,y_{i+1},h_i)   \\
p(y_n|h) & \propto \exp{\mu_\alpha(y_n)} \ .
\end{align*}

In [68]:
def multinomial_vectorized(logp, axis=1):
    sample = np.zeros(logp.shape, dtype=np.int32)
    p = np.exp( logp - np.expand_dims(log_sum_exp( logp, axis=axis), axis) )
    for b in np.arange(p.shape[0]):
        sample[b] = np.random.multinomial(1, p[b])
    return sample

def sampling_ffbs_hot( f, g, time_major=False):
    nu_alp = forward_pass( f, g)
    if not time_major:
        nu_alp = np.transpose(nu_alp, [1, 0, 2])
        g = np.transpose(g, [1, 0, 2, 3])
    sample = np.zeros(np.shape(nu_alp), dtype=np.int32)
    sample[-1] = multinomial_vectorized(nu_alp[-1]) 
    sequence_length = nu_alp.shape[0]
    for index in range(1, sequence_length):
        sample[-index-1] = multinomial_vectorized( 
            np.sum( g[-index]*np.expand_dims(sample[-index], axis=2), axis=2) + nu_alp[-index])
    if not time_major:
        sample = np.transpose(sample, [1, 0, 2])
    return sample

print(sampling_ffbs_hot( f, g)[0,0:3])

[[1 0]
 [1 0]
 [0 1]]


# More examples

## Example 2 

Still independent variables but now with a bias

In [69]:
b_2 = 4 # batch_size
n_2 = 3 # length of sequence
c_2 = 2 # number of classes

g_2 = np.zeros((b_2, n_2-1, c_2, c_2)) # independent variables
f_2 = np.zeros((b_2, n_2, c_2)) #

f_2[:, :, 0] = np.log(3) 
f_2[:, :, 1] = np.log(1) # this should give probability 3 / (3 + 1 ) = 3/4 for class 1

# define some example data that we will use below to calculate log likelihood
y_2 = np.zeros([b_2, n_2], dtype=np.int32) # all label belong to class 1

nu_alp_2 = forward_pass(f_2, g_2)
print("nu_alp")
print(np.exp(nu_alp_2)[0].T)
print()

nu_bet_2 = backward_pass(f_2, g_2)
print("nu_bet")
print(np.exp(nu_bet_2)[0].T)
print()

print("logZ")
print(logZ(nu_alp_2, nu_bet_2)[0]) # should give same log Z no matter what slice we use
print(logZ(nu_alp_2, nu_bet_2, 2)[0])
print(logZ(nu_alp_2, nu_bet_2, n_2-1)[0])
print(n_2*np.log(4)) # exact log Z
print()

print("log_likelihood")
print(log_likelihood(y_2, f_2, g_2, nu_alp_2, nu_bet_2))
print()
print("log_likelihood_vectorized")
print(log_likelihood_vectorized(y_2, f_2, g_2, nu_alp_2, nu_bet_2))
print()
print("compare to above")
print(n_2*np.log(3.0/4.0)) # compare to independent
print()
print("log_marginal")
print(np.exp(log_marginal(nu_alp_2, nu_bet_2))[0].T)

nu_alp
[[  3.  12.  48.]
 [  1.   4.  16.]]

nu_bet
[[ 16.   4.   1.]
 [ 16.   4.   1.]]

logZ
4.15888308336
4.15888308336
4.15888308336
4.15888308336

log_likelihood
-0.863046217355

log_likelihood_vectorized
-0.863046217355

compare to above
-0.863046217355

log_marginal
[[ 0.75  0.75  0.75]
 [ 0.25  0.25  0.25]]


## Example 3 
    
Now with dependent variables that will make class 1 more likely at each position

In [70]:
b_3 = 4 # batch_size
n_3 = 3 # length of sequence
c_3 = 2 # number of classes

g_3 = np.zeros((b_3, n_3-1, c_3, c_3)) # independent variables
f_3 = np.zeros((b_3, n_3, c_3)) #

f_3[:, :, 0] = np.log(3) 
f_3[:, :, 1] = np.log(1) # this should give probability 3 / (3 + 1 ) = 3/4 for class 1

# define some example data that we will use below to calculate log likelihood
y_3 = np.zeros([b_3, n_3], dtype=np.int32) # all label belong to class 1

g_3[:, :, 0, 0] = np.ones([n_3-1])

print(g_3[0, 0])
print(g_3[0, 1])
print()

nu_alp_3 = forward_pass(f_3, g_3)
nu_bet_3 = backward_pass(f_3, g_3)

print("nu_alp")
print(np.exp(nu_alp_3)[0].T)
print()
print("nu_bet")
print(np.exp(nu_bet_3)[0].T)
print()
print("logZ")
print(logZ(nu_alp_3, nu_bet_3)[0]) # should give same log Z no matter what slice we use
print(logZ(nu_alp_3, nu_bet_3, 1)[0])
print(logZ(nu_alp_3, nu_bet_3, n_3-1)[0])
print()
print("compare")
print(np.log(np.power(3,n_3)*np.power(np.exp(1),n_3-1)+2*np.power(3,n_3-1)*np.power(np.exp(1),n_3-2)+np.power(3,n_3-1)+3*np.power(3,n_3-2)+1)) # exact log Z
print()
print("log_likelihood")
print(log_likelihood(y_3, f_3, g_3, nu_alp_3, nu_bet_3))
print()
print("log_likelihood_vectorized")
print(log_likelihood_vectorized(y_3, f_3, g_3, nu_alp_3, nu_bet_3))
print()
print("compare")
print(n_3*np.log(3.0/4.0)) # compare to independent
print()
print("exp log_marginal")
print(np.exp(log_marginal(nu_alp_3, nu_bet_3))[0].T)
print()
print("Samples")
print(sampling_ffbs_hot(f_3, g_3))

[[ 1.  0.]
 [ 0.  0.]]
[[ 1.  0.]
 [ 0.  0.]]

nu_alp
[[   3.           27.46453646  235.96905113]
 [   1.            4.           31.46453646]]

nu_bet
[[ 78.65635038   9.15484549   1.        ]
 [ 31.46453646   4.           1.        ]]

logZ
5.5888712648
5.5888712648
5.5888712648

compare
5.5888712648

log_likelihood
-0.293034398791

log_likelihood_vectorized
-0.293034398791

compare
-0.863046217355

exp log_marginal
[[ 0.88234635  0.94017206  0.88234635]
 [ 0.11765365  0.05982794  0.11765365]]

Samples
[[[1 0]
  [1 0]
  [1 0]]

 [[1 0]
  [1 0]
  [1 0]]

 [[1 0]
  [1 0]
  [1 0]]

 [[1 0]
  [1 0]
  [1 0]]]


# Example 4

Anto-correlated dependent variables and likelihood evaluated for different sequences

In [71]:
b_4 = 8 # batch_size
n_4 = 3 # length of sequence
c_4 = 2 # number of classes

g_4 = np.zeros((b_4, n_4-1, c_4, c_4)) # independent variables
f_4 = np.zeros((b_4, n_4, c_4)) #

g_4[:, :, 0, 0] = -np.ones([n_4-1]) # no longer independent variables

print(g_4[0, 0])
print(g_4[0, 1])
print()

f_4[:, :, 0] = np.log(3) 
f_4[:, :, 1] = np.log(1) # this should give probability 3 / (3 + 1 ) = 3/4 for class 1

# define some example data that we will use below to calculate log likelihood
y_4 = np.zeros([b_4, n_4], dtype=np.int32) # all label belong to class 1
y_4[1, :] = [0, 0, 1]
y_4[2, :] = [0, 1, 0]
y_4[3, :] = [0, 1, 1]
y_4[4, :] = [1, 0, 0]
y_4[5, :] = [1, 0, 1]
y_4[6, :] = [1, 1, 0]
y_4[7, :] = [1, 1, 1]

y_4_hot = onehot_tensor(y_4, 2)

nu_alp_4 = forward_pass(f_4, g_4)
nu_bet_4 = backward_pass(f_4, g_4)

print("nu_alp")
print(np.exp(nu_alp_4)[0].T)
print()
print("nu_bet")
print(np.exp(nu_bet_4)[0].T)
print()
print("logZ")
print(logZ(nu_alp_4, nu_bet_4)[0]) # should give same log Z no matter what slice we use
print(logZ(nu_alp_4, nu_bet_4, 1)[0])
print(logZ(nu_alp_4, nu_bet_4, n_4-1)[0])
print()
#print("compare")
#print(np.log(np.power(3,n_3)*np.power(np.exp(1),n_3-1)+2*np.power(3,n_3-1)*np.power(np.exp(1),n_3-2)+np.power(3,n_3-1)+3*np.power(3,n_3-2)+1)) # exact log Z
print()
print("log_likelihood")
print(log_likelihood(y_4, f_4, g_4, nu_alp_4, nu_bet_4))
print()
print("log_likelihood_vectorized")
print(log_likelihood_vectorized(y_4, f_4, g_4, nu_alp_4, nu_bet_4))
print()
print("log_likelihood_hot")
print(log_likelihood_hot(y_4_hot, f_4, g_4, nu_alp_4, nu_bet_4))
print()
print("the probability for each possible sequence")
print(np.exp(log_likelihood_hot(y_4_hot, f_4, g_4, nu_alp_4, nu_bet_4, mean_batch=False)))
print()
print("check that the probability add up to one")
print(np.sum(np.exp(log_likelihood_hot(y_4_hot, f_4, g_4, nu_alp_4, nu_bet_4, mean_batch=False))))
print()
print("Viterbi")
print(viterbi(f_4, g_4))
print()
print("exp log_marginal")
print(np.exp(log_marginal(nu_alp_4, nu_bet_4))[0].T)
print()
print("Samples")
print(sampling_ffbs_hot(f_4, g_4)[0:2])

[[-1.  0.]
 [ 0.  0.]]
[[-1.  0.]
 [ 0.  0.]]

nu_alp
[[  3.           6.31091497  18.96496762]
 [  1.           4.          10.31091497]]

nu_bet
[[  6.32165587   2.10363832   1.        ]
 [ 10.31091497   4.           1.        ]]

logZ
3.37676405723
3.37676405723
3.37676405723


log_likelihood
-2.22884562422

log_likelihood_vectorized
-2.22884562422

log_likelihood_hot
-2.22884562422

the probability for each possible sequence
[ 0.12481443  0.1130936   0.30742028  0.10247343  0.1130936   0.10247343
  0.10247343  0.03415781]

check that the probability add up to one
1.0

Viterbi
[[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]

exp log_marginal
[[ 0.64780174  0.45347506  0.64780174]
 [ 0.35219826  0.54652494  0.35219826]]

Samples
[[[0 1]
  [0 1]
  [1 0]]

 [[0 1]
  [1 0]
  [1 0]]]


# Example 5
A small example to test that max marginal and Viterbi is not always the same

In [72]:
b_5 = 4 # batch_size
n_5 = 2 # length of sequence
c_5 = 2 # number of classes

g_5 = np.zeros((b_5, n_5-1, c_5, c_5)) # independent variables
f_5 = np.zeros((b_5, n_5, c_5)) #

g_5[:, :, 0, 0] = -np.ones([n_5-1])
f_5[:, :, 0] = np.log(2)

y_5 = np.zeros([b_5, n_5], dtype=np.int32) # all label belong to class 1
y_5[1, :] = [0, 1]
y_5[2, :] = [1, 0]
y_5[3, :] = [1, 1]

y_5_hot = onehot_tensor(y_5, 2)

nu_alp_5 = forward_pass(f_5, g_5)
nu_bet_5 = backward_pass(f_5, g_5)

print("the probability for each possible sequence")
print(np.exp(log_likelihood_hot(y_5_hot, f_5, g_5, nu_alp_5, nu_bet_5, mean_batch=False)))
print("so the sequences [1,0] and [0,1] are the most probable")
print()
print("Viterbi")
print(viterbi(f_5, g_5))
print("Viterbi finds one of them [1,0]")
print()
print("exp log_marginal")
print(np.exp(log_marginal(nu_alp_5, nu_bet_5))[0].T)
print("According to marginals probability [0,0] should be preferred")
print()
print("Samples")
print(sampling_ffbs_hot(f_5, g_5))

the probability for each possible sequence
[ 0.22738372  0.30904651  0.30904651  0.15452326]
so the sequences [1,0] and [0,1] are the most probable

Viterbi
[[1 0]
 [1 0]
 [1 0]
 [1 0]]
Viterbi finds one of them [1,0]

exp log_marginal
[[ 0.53643023  0.53643023]
 [ 0.46356977  0.46356977]]
According to marginals probability [0,0] should be preferred

Samples
[[[1 0]
  [0 1]]

 [[0 1]
  [0 1]]

 [[1 0]
  [0 1]]

 [[0 1]
  [1 0]]]


## Example 6

Some short sequences with f and g random.

In [73]:
b_6 = 8 # batch_size
n_6 = 3 # length of sequence
c_6 = 2 # number of classes

g_6 = np.zeros((b_6, n_6-1, c_6, c_6)) # independent variables
f_6 = np.zeros((b_6, n_6, c_6)) #
  
# define some example data that we will use below to calculate log likelihood
y_6 = np.zeros([b_6, n_6], dtype=np.int32) # all label belong to class 1
y_6[1, :] = [0, 0, 1]
y_6[2, :] = [0, 1, 0]
y_6[3, :] = [0, 1, 1]
y_6[4, :] = [1, 0, 0]
y_6[5, :] = [1, 0, 1]
y_6[6, :] = [1, 1, 0]
y_6[7, :] = [1, 1, 1]

print("the data")
print(y_6)
print()
    
y_6_hot = onehot_tensor(y_6, 2)
        
np.random.seed(42)

for i in np.arange(5):
    
    print('run ' + repr(i))
    print()
    
    f_6[:] = np.random.randn(n_6, c_6) 
    g_6[:] = np.random.randn(n_6-1, c_6, c_6)  #np.zeros((n_6-1, c_6, c_6)) #np.random.randn(n_6-1, c_6, c_6) 
    
    #print(f_6)
    #print(g_6)
   
    nu_alp_6 = forward_pass(f_6, g_6)
    nu_bet_6 = backward_pass(f_6, g_6)

    print("the probability for each possible sequence")
    print(np.exp(log_likelihood_hot(y_6_hot, f_6, g_6, nu_alp_6, nu_bet_6, mean_batch=False)))
    print()
    print("check that the probability add up to one")
    print(np.sum(np.exp(log_likelihood_hot(y_6_hot, f_6, g_6, nu_alp_6, nu_bet_6, mean_batch=False))))
    print()
    print("Viterbi")
    print(viterbi(f_6, g_6))
    print()
    print("exp log_marginal")
    print(np.exp(log_marginal(nu_alp_6, nu_bet_6))[0].T)
    print()
  

the data
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]

run 0

the probability for each possible sequence
[ 0.17983768  0.17942531  0.3879971   0.04495993  0.01228512  0.01225695
  0.16420977  0.01902813]

check that the probability add up to one
1.0

Viterbi
[[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]

exp log_marginal
[[ 0.79222002  0.38380506  0.74432967]
 [ 0.20777998  0.61619494  0.25567033]]

run 1

the probability for each possible sequence
[ 0.14960726  0.17399722  0.0566593   0.15750672  0.11821717  0.13748971
  0.05463718  0.15188544]

check that the probability add up to one
1.0

Viterbi
[[0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]

exp log_marginal
[[ 0.53777051  0.57931136  0.37912092]
 [ 0.46222949  0.42068864  0.62087908]]

run 2

the probability for each possible sequence
[ 0.03555296  0.05750414  0.42338088  0.08451763  0.02621383  0.04239882
  0.27544574  0.054986  ]

check that the pr

# CRF as latent layer

Let us define a latent layer in RNN type model which is a CRF and a likelihood for a sequence $y$ using the following model:
\begin{align}
z & \sim {\rm CRF}(z|h) \\
y & \sim p(y|h,z) = \prod_i^n p(y_i|h_i,z_i) \ .
\end{align}
This model still has a CRF structure and we can perform exact marginalization. If we denote the normalizer computed with factors $f$ and $g$ by $Z(f,g)$ then we get:
\begin{align}
p(y|h) = \sum_z p(y|h,z) {\rm CRF}(z|h) = \frac{Z(f+\log p(y|z,h),g)}{Z(f,g)} \ .
\end{align}
To compute this likelihood we thus need to perform two forward passes for the graphs defined by $(f,g)$ and $(f+\log p(y|z,h),g)$.

For prediction we note that the graph with both $z$ and $y$ unobserved still defines a tree and exact computation of marginals $p(z_i|h)$ and $p(y_i|h)$ and Viterbi decoding ${\rm argmax}_{y,z} p(y,z|h)$ are therefore possible. Note however that the more interesting decoding ${\rm argmax}_{y} p(y|h)$ is not possible because $p(y|h)$ is not a tree. The details will appear later. 

## Example 7

Softmax output layer example

In [74]:
def log_softmax(z, axis=0):
    return z - np.expand_dims(log_sum_exp(z, axis), axis=axis)

b_7 = 8 # batch_size
n_7 = 3 # length of sequence
c_7 = 4 # number of latent classes
c_out_7 = 2 # number of classes


g_7 = np.zeros((b_7, n_7-1, c_7, c_7)) # independent variables
f_7 = np.zeros((b_7, n_7, c_7)) #
w_7 = np.random.randn(c_out_7,c_7) # the weights the output with the latent
  
# define some example data that we will use below to calculate log likelihood
y_7 = np.zeros([b_7, n_7], dtype=np.int32) # all label belong to class 1
y_7[1, :] = [0, 0, 1]
y_7[2, :] = [0, 1, 0]
y_7[3, :] = [0, 1, 1]
y_7[4, :] = [1, 0, 0]
y_7[5, :] = [1, 0, 1]
y_7[6, :] = [1, 1, 0]
y_7[7, :] = [1, 1, 1]

print("the data")
print(y_7)
print()
    
y_7_hot = onehot_tensor(y_7, 2)
        
np.random.seed(42)

for i in np.arange(5):
    
    print('run ' + repr(i))
    print()
    
    f_7[:] = np.random.randn(n_7, c_7) 
    g_7[:] = np.random.randn(n_7-1, c_7, c_7)  #np.zeros((n_6-1, c_6, c_6)) #np.random.randn(n_6-1, c_6, c_6) 
    f_out_7 = np.dot(y_7_hot, log_softmax(w_7))
    
    nu_alp_7 = forward_pass(f_7, g_7) # forward backward to get Z
    nu_bet_7 = backward_pass(f_7, g_7)
    
    nu_out_alp_7 = forward_pass(f_7 + f_out_7, g_7) # forward backward to get Z_out
    nu_out_bet_7 = backward_pass(f_7 + f_out_7, g_7)

    log_like = logZ(nu_out_alp_7 , nu_out_bet_7) - logZ(nu_alp_7 , nu_bet_7)
    
    mar = np.exp(log_marginal(nu_out_alp_7, nu_out_bet_7))
    mar_out = np.exp(log_softmax( np.dot(mar, np.transpose(w_7)), axis=2))

    #print("softmax probabilities")
    #print(np.exp(log_softmax(w_7)))
    #print()
    print("the probability for each possible sequence")
    print(np.exp(log_like))
    print()
    print("check that the probability add up to one")
    print(np.sum(np.exp(log_like)))
    print()
    print("marginals latent")
    print(mar[0])
    print()
    print("marginals out")
    print(mar_out[0])
    print()


the data
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]

run 0

the probability for each possible sequence
[ 0.45186689  0.1355856   0.11416276  0.03217685  0.16506251  0.04482619
  0.04506487  0.01125433]

check that the probability add up to one
1.0

marginals latent
[[ 0.15555653  0.03678695  0.21949861  0.58815792]
 [ 0.3395483   0.0667786   0.11347221  0.48020089]
 [ 0.2776246   0.28757036  0.07996116  0.35484387]]

marginals out
[[ 0.82624296  0.17375704]
 [ 0.87446722  0.12553278]
 [ 0.843573    0.156427  ]]

run 1

the probability for each possible sequence
[ 0.40045335  0.20990225  0.10920411  0.08065662  0.10474243  0.05170418
  0.02513555  0.01820152]

check that the probability add up to one
1.0

marginals latent
[[ 0.03788099  0.02809253  0.10356142  0.83046506]
 [ 0.4577009   0.00694176  0.16712286  0.36823448]
 [ 0.03763978  0.30786458  0.27705213  0.37744351]]

marginals out
[[ 0.84131149  0.15868851]
 [ 0.88444029  0.11555971]
 [ 0.7314639   0

# From autoregressive to CRF 

The causal autoregressive first order Markov model is
\begin{align}
p(y|h) = p(y_1|h_1)\prod_{i=2}^n p(y_i|y_{i-1},h_i) \ .
\end{align}
Each term is given by a softmax
\begin{align}
p(y_1|h_1) & = \frac{\exp( w_0(y_1,h_1))}{\sum_{y} \exp( w_0(y,h_i)) } \\
p(y_i|y_{i-1},h_i) & = \frac{\exp( w(y_{i-1},y_{i},h_{i-1}))}{\sum_{y} \exp( w(y_{i-1},y,h_{i-1})) } \ .
\end{align}
We can now compare this with the definition of the CRF and translate to the CRF definition. Defining the indicator
function $I({\rm true})=1$ and $I({\rm false})=0$ we can write:
\begin{align}
f(y_i,h_i) & = I(i=1) w_0(y_1,h_1) - I(i<n) \log \sum_{y} \exp( w(y_i,y,h_i)) \\
g(y_i,y_{i+1},h_i) & = w(y_{i},y_{i+1},h_{i}) \ .
\end{align}
So the autoregressive model is nothing but a constraint version of the CRF.

In [75]:
# the weight matrix w is laid out like g: b x n-1 x c x c 
# the weight w0 is: b x c

def ar_to_crf(w0, w):
    # right now w needs to be 4 dim, could be nice with a version where it could be 2 or 3 dim. 
    fs = - log_sum_exp(w, np.ndim(w)-2)
    fs[:,0] = fs[:,0] + w0
    f = np.zeros(tuple(map(sum, zip(np.shape(fs), (0,1,0))))) # add one to length of f
    f[:,:-1]=fs 
    return (f,w)

In [76]:
## Example 7.1

w_7_1 = np.zeros(np.shape(g_7))
w0_7_1 = np.ones((b_7,c_7))

(f_7_1,g_7_1) = ar_to_crf(w0_7_1,w_7_1)

print("w0")
print(w0_7_1[0])
print()
print("w")
print(w_7_1[0])
print()
print("f")
print(f_7_1[0])
print()
print("g")
print(g_7_1[0])
print()



w0
[ 1.  1.  1.  1.]

w
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]

f
[[-0.38629436 -0.38629436 -0.38629436 -0.38629436]
 [-1.38629436 -1.38629436 -1.38629436 -1.38629436]
 [ 0.          0.          0.          0.        ]]

g
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]



# Not fully observed data

We can also handle the situation where the time series is only observed at $n_o\leq n$ timepoints 
$(t_1,\ldots,t_{n_o}$. The model is now
\begin{align}
z & \sim {\rm CRF}(z|h) \\
y_o & \sim p(y_o|h,z) = \prod_i^{n_o} p(y_{t_i}|h_{t_i},z_{t_i}) \ .
\end{align}
We can perform exact inference as before and infer latent state and marginal for all timepoints.

In [77]:
b_8 = 4 # batch_size
n_8 = 10 # length of sequence
c_8 = 3 # number of latent classes
c_out_8 = 2 # number of classes

g_8 = np.zeros((b_8, n_8-1, c_8, c_8)) # independent variables
f_8 = np.zeros((b_8, n_8, c_8)) #
  
# define some example data that we will use below to calculate log likelihood
y_8 = np.zeros([b_8, n_8], dtype=np.int32) # all label belong to class 1
y_8[1, :] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
y_8[2, :] = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_8[3, :] = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]

y_mask_8 = np.zeros([b_8, n_8], dtype=np.int32) # we observe first and last timepoint
y_mask_8[:] = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]

print("the data - all 4 possible sequences with the first and last symbol observed")
print(y_8)
print()
print("the data mask - mask out everything but first and last symbol")
print(y_mask_8)
print()
    
y_8_hot = onehot_tensor(y_8, 2)
        
persist = 1000
g_8[:] = (persist+1)*np.eye(c_8,c_8) - persist*np.ones((c_8,c_8)) # np.random.randn(n_7-1, c_7, c_7)
    
w_8 = (persist+1)*np.eye(c_out_8,c_8) - persist*np.ones((c_out_8,c_8)) #np.zeros((c_out_8,c_8)) # the weights the output with the latent

f_out_8 = np.dot(y_8_hot*np.expand_dims(y_mask_8, 2), log_softmax(w_8))
    
nu_alp_8 = forward_pass(f_8, g_8) # forward backward to get Z
nu_bet_8 = backward_pass(f_8, g_8)
    
nu_out_alp_8 = forward_pass(f_8 + f_out_8, g_8) # forward backward to get Z_out
nu_out_bet_8 = backward_pass(f_8 + f_out_8, g_8)

log_like = logZ(nu_out_alp_8 , nu_out_bet_8) - logZ(nu_alp_8 , nu_bet_8)
    
mar = np.exp(log_marginal(nu_out_alp_8, nu_out_bet_8))
mar_out = np.exp(log_softmax( np.dot(mar, np.transpose(w_8)), axis=2))

print("softmax probabilities")
print(np.exp(log_softmax(w_8)))
print()
print("the probability for each possible sequence")
print(np.exp(log_like))
print()
print("check that the probability add up to one")
print(np.sum(np.exp(log_like)))
print()

for i in np.arange(b_8):
    
    print('data point ' + repr(i))
    print(y_8[i])
    print()
    print("marginals latent")
    print(mar[i])
    print()
    print("marginals out")
    print(mar_out[i])
    print()


the data - all 4 possible sequences with the first and last symbol observed
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1]]

the data mask - mask out everything but first and last symbol
[[1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]]

softmax probabilities
[[ 1.   0.   0.5]
 [ 0.   1.   0.5]]

the probability for each possible sequence
[ 0.41666667  0.08333333  0.08333333  0.41666667]

check that the probability add up to one
1.0

data point 0
[0 0 0 0 0 0 0 0 0 0]

marginals latent
[[ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]
 [ 0.8  0.   0.2]]

marginals out
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]

data point 1
[0 0 0 0 0 0 0 0 0 1]

marginals latent
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]


# CRF decoder with attention

Consider a (bi-directional) encoder that gives a sequence $h$ that feeds into (bi-directional) decoder that gives a sequence $s$.  
One way of implementing a CRF decoder with attention to $h$ is to let the CRF
factors $f$ and $g$ be functions of context vectors $c_i$ and $\hat{c}_i$ the and decoder state $s_i$:
$f(y_i,c_i,s_i)$ and $g(y_i,y_{i+1},\hat{c}_i,s_i)$. We can also let the context vectors depend on the labels: $c_i(y_i)$ and $\hat{c}_i(y_i,y_{i+1})$. 

An important difference to the usual language model is that the decoder cannot use the already emitted symbols because then the RNN would act as an label information channel violating the nearest neighbour only requirement of the CRF. This restriction makes the decoder a much less powerful language model. This will instead be the task of the CRF. The biggest benefit of the CRF is that we can make exact inference.

These are the equations for calculating $f(y_i,c_i,s_i)$. The calculation of $g(y_i,y_{i+1},\hat{c}_i,s_i)$ is completely analogeous and can be found in the code. Step one is to calculate attention weights for each label, decoder position and encoder position tuple:    
$$\alpha_{ij}(y_i) = \alpha_{ij}(y_i,s_i,h)= \frac{\exp(e(y_i,s_i,h_j))}{\sum_{j'}\exp(e(y_i,s_i,h_{j'}))}
\ . 
$$
In the second step we calculate the context vector for each label and decoder position pair:
\begin{align*}
c_i(y_i) & = \sum_{j=1}^{n_{\rm in}} \alpha_{ij}(y_i) \, h_j \ .
\end{align*}
Then finally we compute $f(y_i,c_i,s_i)$. This is implemented as a simple linear function in the code.

In [78]:
# Example 9 - instead of implementing the actual encode-decoder architecture the example just assumes that 
# encoder states h, decoder states s and the attention "error" e is given. Assumes a simple linear f(y_i,c_i,s_i)  
b_9 = 8
c_9 = 2
n_9 = 20
n_in = 10
h_dim = 15
s_dim = 7

e_f = np.random.randn(b_9, c_9, n_9, n_in); # error term in attention softmax for f - b x c x n x n_in
e_g = np.random.randn(b_9, c_9, c_9, n_9-1, n_in); # error term in attention softmax for g - b x c x c x n-1 x n_in
h = np.random.randn(b_9, n_in, h_dim) # output from encoder - b x n_in x h_dim
s = np.random.randn(b_9, n_9, s_dim) # output from decoder - b x n x s_dim
w_fc = np.random.randn(c_9, h_dim) # weights for context verctor - c x h_dim
w_fs = np.random.randn(c_9, s_dim) # weights for decoder - c x s_dim
w_gc = np.random.randn(c_9, c_9, h_dim) # weights for context verctor - c x c x h_dim
w_gs = np.random.randn(c_9, c_9, s_dim) # weights for decoder - c x c x s_dim

alpha_f = np.exp(log_softmax(e_f, axis=3)) # b x c x n x n_in 
ctxt_f = np.sum(np.expand_dims(alpha_f, 4) * np.reshape(h, (b_9, 1, 1, n_in, h_dim)), axis=3) # b x c x n x h_dim
f_9 = np.swapaxes(np.sum( ctxt_f * np.reshape(w_fc, (1, c_9, 1, h_dim)), axis=3), 1, 2) + np.dot(s, np.transpose(w_fs)) # b x n x c

print(np.shape(alpha_f))
print(np.shape(ctxt_f))
print(np.shape(f_9))

alpha_g = np.exp(log_softmax(e_g, axis=4)) # b x c x c x n-1 x n_in
ctxt_g = np.sum(np.expand_dims(alpha_g, 5) * np.reshape(h, (b_9, 1, 1, 1, n_in, h_dim)), axis=4) # b x c x c x n-1 x h_dim
g_9 = np.transpose(np.sum( ctxt_g * np.reshape(w_gc, (1, c_9, c_9, 1, h_dim)), axis=4),[0, 3, 1, 2]) + np.tensordot(s[:,:-1], w_gs, ([2, 2]))

print(np.shape(alpha_g))
print(np.shape(ctxt_g))
print(np.shape(g_9))

nu_alp_9 = forward_pass(f_9, g_9) # forward backward to get Z
nu_bet_9 = backward_pass(f_9, g_9)
 
print()
print("log Z")
print(logZ(nu_alp_9, nu_bet_9))
print()

(8, 2, 20, 10)
(8, 2, 20, 15)
(8, 20, 2)
(8, 2, 2, 19, 10)
(8, 2, 2, 19, 15)
(8, 19, 2, 2)

log Z
[ 95.0911613   84.36973226  60.13453541  70.4113463   84.07281802
  78.76179297  56.69576373  98.13610992]

